In [ ]:
! pip install pandas numpy matplotlib seaborn statsmodels scikit-learn prophet

In [ ]:
# load data
import pandas as pd

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url, parse_dates=['Month'], index_col='Month')
print(df.head())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.index, df['Passengers'])
plt.title('Airline Passengers Over Time')
plt.xlabel('Date')
plt.ylabel('Passengers')
plt.show()

In [ ]:
# prepare data
print(df.isnull().sum())
df.fillna(method='ffill', inplace=True)

In [ ]:
# analyze trends and seasonality
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df['Passengers'], model='additive')
decomposition.plot()
plt.show()

In [7]:
# split data
train = df.iloc[:-12]
test = df.iloc[-12:]

In [ ]:
# apply ARIMA (AutoRegressive Integrated Moving Average) model
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(train, order=(5,1,0))
model_fit = model.fit()
print(model_fit.summary())

# make predictions
forecast = model_fit.forecast(steps=12)
plt.plot(test.index, test['Passengers'], label='Actual')
plt.plot(test.index, forecast, label='Forecast')
plt.legend()
plt.show()


In [ ]:
# use Prophet for handling seasonality and trends
from prophet import Prophet
df.reset_index(inplace=True)
df.rename(columns={'Month': 'ds', 'Passengers': 'y'}, inplace=True)
model = Prophet()
model.fit(df[:-12])

future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)
model.plot(forecast)
plt.show()

In [ ]:
# calculate evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

mae = mean_absolute_error(test['Passengers'], forecast['yhat'].tail(12))
mse = mean_squared_error(test['Passengers'], forecast['yhat'].tail(12))
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')